In [1]:
# data_preprocessing.py
"""
Reads "Cisco Forecast League Clean.xlsx" containing:
  - "Main Forecast Data": 20 unique products with historical actuals (FY22 Q2–FY25 Q1)
    and team forecasts for FY25 Q2.
  - "Accuracy Data": Each team's Accuracy and Bias for FY25 Q1, FY24 Q4, and FY24 Q3,
    stored as strings like "79.46%".
This module:
  - Removes duplicates.
  - Normalizes and converts Accuracy/Bias columns from percentage strings to decimals
    (without dividing by 100, since Excel stores as decimals).
  - Saves wide-format CSVs (main_forecast_data.csv and accuracy_data.csv).
  - Reshapes historical actuals into long format (time_series_data.csv) using only historical actuals,
    i.e., excluding the static team forecast columns.
  - Aggregates team performance (average bias & accuracy per product/team) and saves as team_performance.csv.
"""

import pandas as pd
import sys
import re

def normalize_columns(df):
    """Normalize column names by replacing multiple spaces with a single space and stripping."""
    df.columns = df.columns.str.replace(r'\s+', ' ', regex=True).str.strip()
    return df

def convert_percentage_columns(df, columns):
    """
    Converts percentage columns (with '%' symbol) to decimals.
    Assumes the underlying value is already a decimal (e.g., "0.7946%" becomes 0.7946).
    """
    for col in columns:
        df[col] = df[col].astype(str).str.replace('%', '', regex=False).str.strip()
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

def preprocess_data(excel_file="Cisco Forecast League Clean.xlsx"):
    try:
        main_df = pd.read_excel(excel_file, sheet_name="Main Forecast Data")
        acc_df = pd.read_excel(excel_file, sheet_name="Accuracy Data", dtype={"Product Name": str})
        print("Excel file loaded successfully.")
    except Exception as e:
        sys.exit(f"Error reading Excel file: {e}")

    # Normalize column names in acc_df
    acc_df = normalize_columns(acc_df)

    # Remove duplicate products in main_df
    main_df = main_df.drop_duplicates(subset=["Product Name"], keep="first")
    if main_df["Product Name"].nunique() != 20:
        print(f"WARNING: Found {main_df['Product Name'].nunique()} unique products instead of 20.")
    else:
        print("Confirmed 20 unique products in main data.")

    # Save main forecast data (wide format)
    try:
        main_df.to_csv("main_forecast_data.csv", index=False)
        print("Saved main_forecast_data.csv.")
    except Exception as e:
        sys.exit(f"Error saving main_forecast_data.csv: {e}")

    # Convert accuracy/bias columns in acc_df
    acc_bias_cols = [
        col for col in acc_df.columns
        if ("Accuracy" in col or "Bias" in col) and col not in ["Product Name", "Cost Rank"]
    ]
    acc_df = convert_percentage_columns(acc_df, acc_bias_cols)
    try:
        acc_df.to_csv("accuracy_data.csv", index=False)
        print("Saved accuracy_data.csv with converted decimals.")
    except Exception as e:
        sys.exit(f"Error saving accuracy_data.csv: {e}")

    # Reshape historical actuals into long format.
    # Exclude the static team forecast columns.
    historical_quarters = [
        "FY22 Q2", "FY22 Q3", "FY22 Q4",
        "FY23 Q1", "FY23 Q2", "FY23 Q3", "FY23 Q4",
        "FY24 Q1", "FY24 Q2", "FY24 Q3", "FY24 Q4",
        "FY25 Q1"
    ]
    id_vars = [
        "Cost Rank", "Product Name", "Product Life Cycle", "Your Forecast FY25 Q2"
    ]
    try:
        ts_df = main_df.melt(
            id_vars=id_vars,
            value_vars=historical_quarters,
            var_name="Fiscal_Quarter",
            value_name="Actual_Units"
        )
        # Create an ordering column for the quarters
        quarter_order = {q: i for i, q in enumerate(historical_quarters, start=1)}
        ts_df["Quarter_Order"] = ts_df["Fiscal_Quarter"].map(quarter_order)
    except Exception as e:
        sys.exit(f"Error reshaping historical actuals: {e}")

    try:
        ts_df.to_csv("time_series_data.csv", index=False)
        print("Saved time_series_data.csv.")
    except Exception as e:
        sys.exit(f"Error saving time_series_data.csv: {e}")

    # Aggregate team performance (average bias & accuracy) for each product.
    rows = []
    for idx, row_ in acc_df.iterrows():
        product = row_["Product Name"]

        # Demand Planners
        dp_acc_values = []
        dp_bias_values = []
        for quarter in ["FY2025 Q1", "FY24 Q4", "FY24 Q3"]:
            acc_col = f"Demand Planners Accuracy {quarter}"
            bias_col = f"Demand Planners Bias {quarter}"
            # Use .get() and convert to float if possible
            val_acc = row_.get(acc_col)
            val_bias = row_.get(bias_col)
            try:
                if pd.notna(val_acc):
                    dp_acc_values.append(float(val_acc))
                if pd.notna(val_bias):
                    dp_bias_values.append(float(val_bias))
            except Exception as e:
                print(f"Conversion error in Demand Planners for product {product}: {e}")
        dp_avg_acc = sum(dp_acc_values) / len(dp_acc_values) if dp_acc_values else None
        dp_avg_bias = sum(dp_bias_values) / len(dp_bias_values) if dp_bias_values else None

        # Marketing Teams
        mk_acc_values = []
        mk_bias_values = []
        for quarter in ["FY2025 Q1", "FY24 Q4", "FY24 Q3"]:
            acc_col = f"Marketing Teams Accuracy {quarter}"
            bias_col = f"Marketing Teams Bias {quarter}"
            val_acc = row_.get(acc_col)
            val_bias = row_.get(bias_col)
            try:
                if pd.notna(val_acc):
                    mk_acc_values.append(float(val_acc))
                if pd.notna(val_bias):
                    mk_bias_values.append(float(val_bias))
            except Exception as e:
                print(f"Conversion error in Marketing Teams for product {product}: {e}")
        mk_avg_acc = sum(mk_acc_values) / len(mk_acc_values) if mk_acc_values else None
        mk_avg_bias = sum(mk_bias_values) / len(mk_bias_values) if mk_bias_values else None

        # Stat and ML
        stat_acc_values = []
        stat_bias_values = []
        for quarter in ["FY2025 Q1", "FY24 Q4", "FY24 Q3"]:
            acc_col = f"Stat and ML Accuracy {quarter}"
            bias_col = f"Stat and ML Bias {quarter}"
            val_acc = row_.get(acc_col)
            val_bias = row_.get(bias_col)
            try:
                if pd.notna(val_acc):
                    stat_acc_values.append(float(val_acc))
                if pd.notna(val_bias):
                    stat_bias_values.append(float(val_bias))
            except Exception as e:
                print(f"Conversion error in Stat and ML for product {product}: {e}")
        stat_avg_acc = sum(stat_acc_values) / len(stat_acc_values) if stat_acc_values else None
        stat_avg_bias = sum(stat_bias_values) / len(stat_bias_values) if stat_bias_values else None

        rows.append({
            "Product Name": product,
            "DP_Accuracy": dp_avg_acc,
            "DP_Bias": dp_avg_bias,
            "MK_Accuracy": mk_avg_acc,
            "MK_Bias": mk_avg_bias,
            "STAT_Accuracy": stat_avg_acc,
            "STAT_Bias": stat_avg_bias
        })

    perf_df = pd.DataFrame(rows)
    try:
        perf_df.to_csv("team_performance.csv", index=False)
        print("Saved team_performance.csv with average bias & accuracy.")
    except Exception as e:
        sys.exit(f"Error saving team_performance.csv: {e}")

if __name__ == "__main__":
    preprocess_data()


Excel file loaded successfully.
Confirmed 20 unique products in main data.
Saved main_forecast_data.csv.
Saved accuracy_data.csv with converted decimals.
Saved time_series_data.csv.
Saved team_performance.csv with average bias & accuracy.


In [2]:
!pip install tigramite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.6/309.6 kB 5.4 MB/s eta 0:00:00


In [3]:
# feature_engineering.py
"""
Loads time_series_data.csv (generated by data_preprocessing.py) and creates additional features:
  - Lag_1: The previous quarter's Actual_Units for each product.
  - MA_3: A 3-quarter moving average of Actual_Units.
Ensures the Quarter_Order column is preserved.
Saves the resulting dataset as time_series_features.csv.
"""

import pandas as pd
import sys

def engineer_features(ts_file="time_series_data.csv"):
    try:
        ts_df = pd.read_csv(ts_file)
        print(f"Loaded {ts_file} with shape: {ts_df.shape}")
    except Exception as e:
        sys.exit(f"Error loading {ts_file}: {e}")

    # Create Lag_1 feature: previous quarter's value per product
    try:
        ts_df["Lag_1"] = ts_df.groupby("Product Name")["Actual_Units"].shift(1)
        print("Lag_1 feature created.")
    except Exception as e:
        sys.exit(f"Error creating Lag_1 feature: {e}")

    # Create MA_3 feature: 3-quarter moving average of Actual_Units
    try:
        ts_df["MA_3"] = ts_df.groupby("Product Name")["Actual_Units"]\
                             .rolling(3, min_periods=1).mean().reset_index(level=0, drop=True)
        print("MA_3 feature created.")
    except Exception as e:
        sys.exit(f"Error creating MA_3 feature: {e}")

    # Ensure Quarter_Order exists; if missing, re-create it.
    if "Quarter_Order" not in ts_df.columns:
        quarters = ["FY22 Q2", "FY22 Q3", "FY22 Q4",
                    "FY23 Q1", "FY23 Q2", "FY23 Q3", "FY23 Q4",
                    "FY24 Q1", "FY24 Q2", "FY24 Q3", "FY24 Q4",
                    "FY25 Q1"]
        quarter_order = {q: i for i, q in enumerate(quarters, start=1)}
        if "Fiscal_Quarter" in ts_df.columns:
            ts_df["Quarter_Order"] = ts_df["Fiscal_Quarter"].map(quarter_order)
    else:
        print("Quarter_Order column exists.")

    try:
        ts_df.to_csv("time_series_features.csv", index=False)
        print("Saved time_series_features.csv.")
    except Exception as e:
        sys.exit(f"Error saving time_series_features.csv: {e}")

if __name__ == "__main__":
    engineer_features()


Loaded time_series_data.csv with shape: (240, 7)
Lag_1 feature created.
MA_3 feature created.
Quarter_Order column exists.
Saved time_series_features.csv.


In [4]:
# model_training.py
"""
Trains three forecasting models using time_series_features.csv:
  1. Exponential Smoothing (ES)
  2. ARIMA (order (1,1,1))
  3. XGBoost (using Lag_1 and MA_3 features)
Averages the forecasts to produce "Our_Forecast" for each product.
Saves the output as our_forecasts.csv.
"""

import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import sys

def forecast_es(series, steps=1):
    s = series.dropna()
    if len(s) < 4:
        return np.nan
    try:
        model = ExponentialSmoothing(s, trend='add', seasonal='add', seasonal_periods=4)
        fit = model.fit()
        fc = fit.forecast(steps=steps)
        return fc.iloc[-1]
    except Exception as e:
        print(f"ES error: {e}")
        return np.nan

def forecast_arima(series, steps=1, order=(1,1,1)):
    s = series.dropna()
    if len(s) < 4:
        return np.nan
    try:
        model = ARIMA(s, order=order)
        fit = model.fit()
        fc = fit.forecast(steps=steps)
        return fc.iloc[-1]
    except Exception as e:
        print(f"ARIMA error: {e}")
        return np.nan

def train_and_forecast(ts_file="time_series_features.csv"):
    try:
        ts_df = pd.read_csv(ts_file)
        print(f"Loaded {ts_file}, shape={ts_df.shape}")
    except Exception as e:
        sys.exit(f"Error loading {ts_file}: {e}")

    # Forecast using ES and ARIMA per product
    es_forecasts = {}
    arima_forecasts = {}
    for product, group in ts_df.groupby("Product Name"):
        if "Quarter_Order" not in group.columns:
            sys.exit("Error: Quarter_Order column missing in time_series_features.csv")
        group_sorted = group.sort_values("Quarter_Order")
        series = group_sorted["Actual_Units"]
        es_forecasts[product] = forecast_es(series, steps=1)
        arima_forecasts[product] = forecast_arima(series, steps=1, order=(1,1,1))

    es_df = pd.DataFrame(list(es_forecasts.items()), columns=["Product Name", "ES_Forecast"])
    arima_df = pd.DataFrame(list(arima_forecasts.items()), columns=["Product Name", "ARIMA_Forecast"])

    # XGBoost forecast on pooled data using Lag_1 and MA_3
    ml_data = ts_df.dropna(subset=["Lag_1", "MA_3", "Actual_Units"]).copy()
    ml_data["Actual_Units"] = pd.to_numeric(ml_data["Actual_Units"], errors="coerce")
    ml_data = ml_data.dropna(subset=["Actual_Units"])
    features = ["Lag_1", "MA_3"]
    X = ml_data[features]
    y = ml_data["Actual_Units"]

    tscv = TimeSeriesSplit(n_splits=3)
    xgb_model = XGBRegressor(objective="reg:squarederror", random_state=42)
    param_grid = {"max_depth": [3, 5], "learning_rate": [0.1], "n_estimators": [50, 100]}
    try:
        grid_search = GridSearchCV(xgb_model, param_grid, cv=tscv, scoring="neg_mean_absolute_error", n_jobs=-1)
        grid_search.fit(X, y)
        best_xgb = grid_search.best_estimator_
        print("Best XGB params:", grid_search.best_params_)
    except Exception as e:
        sys.exit(f"Error training XGBoost: {e}")

    xgb_preds = {}
    for product, group in ts_df.groupby("Product Name"):
        group_sorted = group.sort_values("Quarter_Order")
        last_row = group_sorted.iloc[-1]
        if pd.isna(last_row["Lag_1"]) or pd.isna(last_row["MA_3"]):
            xgb_preds[product] = np.nan
        else:
            feat_vector = [[last_row["Lag_1"], last_row["MA_3"]]]
            try:
                xgb_preds[product] = best_xgb.predict(feat_vector)[0]
            except Exception as e:
                print(f"XGB prediction error for {product}: {e}")
                xgb_preds[product] = np.nan

    xgb_df = pd.DataFrame(list(xgb_preds.items()), columns=["Product Name", "XGB_Forecast"])

    # Merge forecasts and compute Our_Forecast as average of ES, ARIMA, and XGB forecasts
    merge_df = es_df.merge(arima_df, on="Product Name", how="outer").merge(xgb_df, on="Product Name", how="outer")
    merge_df["Our_Forecast"] = merge_df[["ES_Forecast", "ARIMA_Forecast", "XGB_Forecast"]].mean(axis=1)

    try:
        merge_df.to_csv("our_forecasts.csv", index=False)
        print("Saved our_forecasts.csv.")
    except Exception as e:
        sys.exit(f"Error saving our_forecasts.csv: {e}")

if __name__ == "__main__":
    train_and_forecast()


Loaded time_series_features.csv, shape=(240, 9)


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/usr/local/lib/python3.11/dist-packages/statsmode

ES error: Cannot compute initial seasonals using heuristic method with less than two full seasonal cycles in the data.


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmode

ES error: Cannot compute initial seasonals using heuristic method with less than two full seasonal cycles in the data.


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_pred

Best XGB params: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
Saved our_forecasts.csv.


In [5]:
# final_ensemble.py
"""
Final Ensemble Forecast Module (Enhanced to Reduce Error):
Merges:
  - our_forecasts.csv (contains Our_Forecast for each product)
  - main_forecast_data.csv (provides each team's FY25 Q2 forecast, Cost Rank, and actuals for FY24 Q3, FY24 Q4, FY25 Q1)
  - team_performance.csv (aggregated average bias and accuracy for each product/team)
Enhancements:
  1. Cap bias values to a reasonable range (e.g., [-0.5, +0.5]) to prevent extreme bias corrections.
  2. Tighten the acceptable ratio range for the team ensemble adjustment (e.g., [0.8, 1.2]).
  3. Optionally use median-based aggregation for global team accuracy.
  4. Use a candidate alpha range with a higher lower bound (0.2 to 0.9) to ensure the team forecast gets at least 20% weight.
Steps:
  - Compute Cost_Factor = (max(Cost Rank) - Cost Rank + 1).
  - Compute global (cost-weighted) team accuracy and derive global weights.
  - Cap bias values and apply bias correction to each team's forecast.
  - Compute the Team Ensemble as the weighted sum of bias-corrected forecasts.
  - Adjust Team Ensemble if its ratio to Our_Forecast falls outside [0.8, 1.2].
  - Optimize the blend parameter α over FY24 Q3, FY24 Q4, and FY25 Q1 (candidate range 0.2 to 0.9) via cost-weighted MAPE.
  - Blend the adjusted Team Ensemble with Our_Forecast using the optimized α.
  - Save the final output as Final_Forecasts_FY25Q2.xlsx.
"""

import pandas as pd
import numpy as np
import sys

def cap_bias(bias, cap=0.5):
    """Cap bias to be within [-cap, cap]."""
    if pd.isna(bias):
        return bias
    return max(-cap, min(cap, bias))

def clamp_bias(bias, lower=-0.99, upper=1.0):
    if pd.isna(bias):
        return bias
    return max(lower, min(upper, bias))

def bias_correct_multiplicative(forecast, avg_bias):
    """Apply bias correction with capped bias."""
    if pd.isna(forecast) or pd.isna(avg_bias):
        return forecast
    capped_bias = cap_bias(avg_bias, cap=0.5)
    denom = 1 + capped_bias
    if denom <= 0:
        return 0.0
    return forecast / denom

def weighted_mape(y_true, y_pred, weights):
    """Compute cost-weighted MAPE."""
    y_true, y_pred, weights = np.array(y_true), np.array(y_pred), np.array(weights)
    nonzero = y_true != 0
    ape = np.abs((y_true[nonzero] - y_pred[nonzero]) / y_true[nonzero])
    return np.sum(weights[nonzero] * ape) / np.sum(weights[nonzero])

def evaluate_alpha(final_df, holdout_col, alpha):
    """Compute cost-weighted MAPE for a given alpha using actuals from the specified column."""
    if holdout_col not in final_df.columns:
        return None
    y_true = final_df[holdout_col].dropna().to_numpy()
    if len(y_true) == 0:
        return None
    valid_idx = final_df[holdout_col].notna()
    y_pred = alpha * final_df.loc[valid_idx, "Team_Ensemble"].to_numpy() + (1 - alpha) * final_df.loc[valid_idx, "Our_Forecast"].to_numpy()
    weights = final_df.loc[valid_idx, "Cost_Factor"].fillna(1).to_numpy()
    return weighted_mape(y_true, y_pred, weights)

def cross_validate_alpha(final_df, holdout_quarters, candidate_alphas):
    best_alpha = None
    best_error = np.inf
    for alpha in candidate_alphas:
        errors = []
        for hq in holdout_quarters:
            err = evaluate_alpha(final_df, hq, alpha)
            if err is not None:
                errors.append(err)
        if len(errors) == 0:
            continue
        avg_err = np.mean(errors)
        print(f"Alpha={alpha:.2f}, per-holdout errors={errors}, average error={avg_err:.4f}")
        if avg_err < best_error:
            best_error = avg_err
            best_alpha = alpha
    return best_alpha, best_error

def adjust_team_ensemble(final_df, lower_bound=0.8, upper_bound=1.2):
    """
    For each product, compute the ratio: Team_Ensemble / Our_Forecast.
    If the ratio is outside [lower_bound, upper_bound],
    adjust Team_Ensemble toward Our_Forecast by clipping the ratio.
    """
    adjusted = []
    for idx, row in final_df.iterrows():
        our = row["Our_Forecast"]
        team = row["Team_Ensemble"]
        if pd.isna(our) or pd.isna(team) or our == 0:
            adjusted.append(team)
            continue
        ratio = team / our
        if ratio < lower_bound:
            ratio_adj = lower_bound
        elif ratio > upper_bound:
            ratio_adj = upper_bound
        else:
            ratio_adj = ratio
        adjusted_value = our * ratio_adj
        adjusted.append(adjusted_value)
    final_df["Team_Ensemble_Adjusted"] = adjusted
    return final_df

def create_final_ensemble(alpha=0.5, use_median=False,
                          holdout_quarters=("FY24 Q3", "FY24 Q4", "FY25 Q1"),
                          alpha_candidates=np.linspace(0.2, 0.9, 8),
                          adjust_ensemble=True):
    try:
        our_df = pd.read_csv("our_forecasts.csv")       # Contains Our_Forecast per product
        main_df = pd.read_csv("main_forecast_data.csv")   # Contains team forecasts, Cost Rank, and actuals for holdouts
        perf_df = pd.read_csv("team_performance.csv", dtype={"Product Name": str})
        print("Loaded our_forecasts.csv, main_forecast_data.csv, and team_performance.csv.")
    except Exception as e:
        sys.exit(f"Error loading CSV files: {e}")

    merge_cols = ["Product Name", "Cost Rank",
                  "Demand Planners Forecast", "Marketing Teams Forecast", "Statistical and ML Forecast"] + list(holdout_quarters)
    final_df = our_df.merge(main_df[merge_cols], on="Product Name", how="left")
    final_df = final_df.merge(perf_df, on="Product Name", how="left")

    try:
        max_cost = final_df["Cost Rank"].max()
    except Exception as e:
        sys.exit(f"Error computing max Cost Rank: {e}")
    final_df["Cost_Factor"] = max_cost - final_df["Cost Rank"] + 1

    total_cost = final_df["Cost_Factor"].sum()
    if use_median:
        global_DP_acc = (final_df["Cost_Factor"] * final_df["DP_Accuracy"]).median() / (final_df["Cost_Factor"].median() or 1)
        global_MK_acc = (final_df["Cost_Factor"] * final_df["MK_Accuracy"]).median() / (final_df["Cost_Factor"].median() or 1)
        global_STAT_acc = (final_df["Cost_Factor"] * final_df["STAT_Accuracy"]).median() / (final_df["Cost_Factor"].median() or 1)
    else:
        global_DP_acc = (final_df["Cost_Factor"] * final_df["DP_Accuracy"]).sum() / total_cost
        global_MK_acc = (final_df["Cost_Factor"] * final_df["MK_Accuracy"]).sum() / total_cost
        global_STAT_acc = (final_df["Cost_Factor"] * final_df["STAT_Accuracy"]).sum() / total_cost

    total_global = global_DP_acc + global_MK_acc + global_STAT_acc
    if total_global == 0:
        global_DP_weight = global_MK_weight = global_STAT_weight = 1/3
    else:
        global_DP_weight = global_DP_acc / total_global
        global_MK_weight = global_MK_acc / total_global
        global_STAT_weight = global_STAT_acc / total_global

    print("Global Weights (cost-weighted):")
    print(f"  Demand Planners: {global_DP_weight:.3f}")
    print(f"  Marketing Teams: {global_MK_weight:.3f}")
    print(f"  Stat and ML: {global_STAT_weight:.3f}")

    final_df["DP_Corrected"] = final_df.apply(lambda row: bias_correct_multiplicative(row["Demand Planners Forecast"], row["DP_Bias"]), axis=1)
    final_df["MK_Corrected"] = final_df.apply(lambda row: bias_correct_multiplicative(row["Marketing Teams Forecast"], row["MK_Bias"]), axis=1)
    final_df["STAT_Corrected"] = final_df.apply(lambda row: bias_correct_multiplicative(row["Statistical and ML Forecast"], row["STAT_Bias"]), axis=1)

    final_df["Team_Ensemble"] = (
        global_DP_weight * final_df["DP_Corrected"] +
        global_MK_weight * final_df["MK_Corrected"] +
        global_STAT_weight * final_df["STAT_Corrected"]
    )

    if adjust_ensemble:
        final_df = adjust_team_ensemble(final_df, lower_bound=0.8, upper_bound=1.2)
        team_ensemble_col = "Team_Ensemble_Adjusted"
    else:
        team_ensemble_col = "Team_Ensemble"

    best_alpha, best_error = cross_validate_alpha(final_df, holdout_quarters, alpha_candidates)
    if best_alpha is None:
        best_alpha = alpha
        print(f"No valid holdout data found. Using alpha={alpha:.2f}")
    print(f"Selected alpha={best_alpha:.2f} with average cost-weighted error={best_error:.4f}")

    final_df["Final_Forecast_FY25Q2"] = (
        best_alpha * final_df[team_ensemble_col] + (1 - best_alpha) * final_df["Our_Forecast"]
    )

    print("Diagnostics (averages):")
    print("  Our_Forecast:", final_df["Our_Forecast"].mean())
    print("  Team Ensemble (before adjustment):", final_df["Team_Ensemble"].mean())
    if adjust_ensemble:
        print("  Team Ensemble (adjusted):", final_df["Team_Ensemble_Adjusted"].mean())
    for hq in holdout_quarters:
        if hq in final_df.columns:
            print(f"  {hq} (Avg Actual):", final_df[hq].dropna().mean())
    print("  Final_Forecast_FY25Q2:", final_df["Final_Forecast_FY25Q2"].mean())

    final_df = final_df.drop_duplicates(subset=["Product Name"], keep="first").reset_index(drop=True)
    try:
        final_df.to_excel("Final_Forecasts_FY25Q2.xlsx", index=False)
        print("Saved Final_Forecasts_FY25Q2.xlsx with final forecast (including Cost Rank).")
    except Exception as e:
        sys.exit(f"Error saving Final_Forecasts_FY25Q2.xlsx: {e}")

if __name__ == "__main__":
    create_final_ensemble(
        alpha=0.5,
        use_median=False,
        holdout_quarters=("FY24 Q3", "FY24 Q4", "FY25 Q1"),
        alpha_candidates=np.linspace(0.2, 0.9, 8),
        adjust_ensemble=True
    )


Loaded our_forecasts.csv, main_forecast_data.csv, and team_performance.csv.
Global Weights (cost-weighted):
  Demand Planners: 0.393
  Marketing Teams: 0.286
  Stat and ML: 0.321
Alpha=0.20, per-holdout errors=[np.float64(0.2369027290886534), np.float64(0.3915288173428199), np.float64(0.11374289760139195)], average error=0.2474
Alpha=0.30, per-holdout errors=[np.float64(0.27010352841522245), np.float64(0.4432964771248301), np.float64(0.12667279018110053)], average error=0.2800
Alpha=0.40, per-holdout errors=[np.float64(0.30548530290563974), np.float64(0.49698291807710065), np.float64(0.14225741117977403)], average error=0.3149
Alpha=0.50, per-holdout errors=[np.float64(0.340867077396057), np.float64(0.557009364586311), np.float64(0.15784203217844742)], average error=0.3519
Alpha=0.60, per-holdout errors=[np.float64(0.3762488518864744), np.float64(0.6177770589533276), np.float64(0.17342665317712092)], average error=0.3892
Alpha=0.70, per-holdout errors=[np.float64(0.4116306263768917), n